<a href="https://colab.research.google.com/github/Ashthespycodes/LBVS-Ligand-Based-Virtual-Screening/blob/main/lbvs_3d_docking_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LBVS Workflow with 2D, 3D Pharmacophore, and Docking
This Colab notebook implements:
1. Excel compound library upload
2. 2D Morgan fingerprints
3. 3D pharmacophore (USRCAT) fingerprints
4. Similarity screening (2D & 3D)
5. Docking-based screening via GNINA
6. ADMET (Lipinski) filtering and scaffold analysis
7. Results saving and visualizations

In [ ]:
# Install RDKit (if needed)
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 15.2 MB/s eta 0:00:00


In [ ]:
# Upload files in Colab
from google.colab import files
import os

print("Upload your Excel compound library (.xlsx/.xls):")
uploaded = files.upload()
input_file = list(uploaded.keys())[0]
print("Compound file loaded:", input_file)

print("Upload your receptor PDB for docking:")
uploaded2 = files.upload()
receptor_file = list(uploaded2.keys())[0]
print("Receptor file loaded:", receptor_file)

Upload your Excel compound library (.xlsx/.xls):


Saving Dataset_carotene+carotenoids(1072).xlsx to Dataset_carotene+carotenoids(1072) (1).xlsx
Compound file loaded: Dataset_carotene+carotenoids(1072) (1).xlsx
Upload your receptor PDB for docking:


Saving 3ert.pdb1.gz to 3ert.pdb1 (1).gz
Receptor file loaded: 3ert.pdb1 (1).gz


In [ ]:
# in a Colab cell, before any rdkit imports:
!pip install -q condacolab
import condacolab
condacolab.install()

# Then install RDKit from conda-forge:
!mamba install -y -c conda-forge rdkit


✨🍰✨ Everything looks OK!

Looking for: ['rdkit']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [ ]:
# Make sure you’ve installed RDKit first:
# !pip install rdkit-pypi

import os
from rdkit import Chem
from rdkit.Chem import AllChem, rdShapeHelpers

# Utility to embed & optimize a 3D conformer
def embed_optimize(smiles, seed=42):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES: {smiles}")
    mol = Chem.AddHs(mol)
    if AllChem.EmbedMolecule(mol, randomSeed=seed) != 0:
        raise RuntimeError("3D embedding failed")
    AllChem.UFFOptimizeMolecule(mol)
    return mol

# Ensure output dir
output_dir = "LBVS_Results"
os.makedirs(output_dir, exist_ok=True)

# Correct β‑carotene SMILES (complete)
query_smiles = (
    "CC1=C(/C=C/C(C)=C/C=C/C(C)=C/C=C/C=C(C)/C=C/"
    "C=C(C)/C=C/C2=C(C)CCCC2(C)C)C(C)(C)CCC1"
)
target_smiles = "C1CCCCC1"  # example; replace with your library SMILES

# Generate 3D conformers
qs = embed_optimize(query_smiles)
ms = embed_optimize(target_smiles)

# Compute shape‐Tanimoto (0=perfect overlap, 1=no overlap)
dist = rdShapeHelpers.ShapeTanimotoDist(qs, ms)
shape_sim = 1.0 - dist
print(f"3D Shape similarity = {shape_sim:.3f}")


3D Shape similarity = 0.085


In [ ]:
# 1. Load Excel dataset and detect SMILES column
df = pd.read_excel(input_file)
smiles_col = None
for col in df.columns:
    if str(col).strip().lower() == "smiles":
        smiles_col = col
        break
if smiles_col is None:
    raise ValueError("No SMILES column found.")
print(f"Using SMILES column: {smiles_col} ({len(df)} compounds)")

Using SMILES column: smiles (1072 compounds)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import GetUSRCAT  # ← import the USRCAT function

query_smiles = {
    "beta_carotene": "CC1=C(/C=C/C(C)=C/C=C/C(C)=C/C=C/C=C(C)/C=C/C=C(C)/C=C/C2=C(C)CCCC2(C)C)C(C)(C)CCC1",
    "bexarotene":    "C=C(c1ccc(C(=O)O)cc1)c1cc2c(cc1C)C(C)(C)CCC2(C)C",
    "tamibarotene":  "CC1(C)CCC(C)(C)c2cc(NC(=O)c3ccc(C(=O)O)cc3)ccc21"
}

query_fps_2d = {}
query_fps_3d = {}

for name, smi in query_smiles.items():
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        raise ValueError(f"Invalid query SMILES: {name}")
    # 2D Morgan fingerprint
    query_fps_2d[name] = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)

    # 3D conformer + USRCAT
    m3d = Chem.AddHs(mol)
    AllChem.EmbedMolecule(m3d, randomSeed=42)
    AllChem.UFFOptimizeMolecule(m3d)
    query_fps_3d[name] = GetUSRCAT(m3d)  # ← now defined!

print("Query fingerprints (2D & 3D USRCAT) computed.")


Query fingerprints (2D & 3D USRCAT) computed.


In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors, rdMolDescriptors
from rdkit.Chem.rdMolDescriptors import GetUSRCAT
import pandas as pd

fps_2d, fps_3d, desc_list, valid_idx = [], [], [], []

for i, smi in enumerate(df[smiles_col]):
    mol = Chem.MolFromSmiles(str(smi))
    if mol is None:
        continue

    # 2D fingerprint
    fps_2d.append(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))

    # 3D conformer + USRCAT
    m3d = Chem.AddHs(mol)
    params = AllChem.ETKDGv3()
    params.randomSeed = 42
    embed_status = AllChem.EmbedMolecule(m3d, params)
    if embed_status != 0 or m3d.GetNumConformers() == 0:
        fps_2d.pop()
        continue

    AllChem.UFFOptimizeMolecule(m3d)
    fps_3d.append(GetUSRCAT(m3d))

    # descriptors
    desc_list.append({
        "MW":   Descriptors.MolWt(mol),
        "LogP": Descriptors.MolLogP(mol),
        "HBD":  Descriptors.NumHDonors(mol),
        "HBA":  Descriptors.NumHAcceptors(mol),
        "TPSA": rdMolDescriptors.CalcTPSA(mol),
        "RotB": Descriptors.NumRotatableBonds(mol)
    })
    valid_idx.append(i)

df_valid = df.iloc[valid_idx].reset_index(drop=True)
df_desc  = pd.DataFrame(desc_list)
df_valid = pd.concat([df_valid, df_desc], axis=1)
print(f"Processed {len(df_valid)} valid molecules.")


Processed 1048 valid molecules.


In [ ]:
# 4. Similarity screening: 2D Tanimoto & 3D cosine similarity
# 2D
sim2d = []
best2d = []
for fp in fps_2d:
    scores = [TanimotoSimilarity(fp, qfp) for qfp in query_fps_2d.values()]
    idx = int(np.argmax(scores))
    sim2d.append(scores[idx])
    best2d.append(list(query_fps_2d.keys())[idx])
# 3D
def cos_sim(a, b): return float(np.dot(a, b) / (np.linalg.norm(a)*np.linalg.norm(b)))
sim3d = []
best3d = []
for v in fps_3d:
    scores = [cos_sim(v, qv) for qv in query_fps_3d.values()]
    idx = int(np.argmax(scores))
    sim3d.append(scores[idx])
    best3d.append(list(query_fps_3d.keys())[idx])

df_valid["Sim2D"] = sim2d
df_valid["Best2D"] = best2d
df_valid["Sim3D"] = sim3d
df_valid["Best3D"] = best3d
threshold = 0.7
df_valid["Hit2D"] = df_valid["Sim2D"] >= threshold
df_valid["Hit3D"] = df_valid["Sim3D"] >= threshold
print(f"2D hits: {df_valid['Hit2D'].sum()}, 3D hits: {df_valid['Hit3D'].sum()}")

2D hits: 108, 3D hits: 1006


In [ ]:
# 5. Docking-based screening (GNINA)
df_valid["DockScore"] = np.nan
df_valid["DockHit"] = False

if shutil.which("gnina"):
    for idx, smi in enumerate(df_valid[smiles_col]):
        lig = Chem.MolFromSmiles(smi)
        lig = Chem.AddHs(lig)
        AllChem.EmbedMolecule(lig, randomSeed=42)
        AllChem.UFFOptimizeMolecule(lig)
        lig_path = f"lig_{idx}.sdf"
        Chem.MolToMolFile(lig, lig_path)
        # Run gnina for score only
        cmd = ["gnina", "--receptor", receptor_file, "--ligand", lig_path,
               "--score_only", "--log", f"log_{idx}.txt"]
        try:
            out = subprocess.check_output(cmd, stderr=subprocess.STDOUT).decode()
            # parse 'Affinity:' line
            for line in out.split("\n"):
                if "Affinity:" in line:
                    score = float(line.split()[1])
                    df_valid.at[idx, "DockScore"] = score
                    df_valid.at[idx, "DockHit"] = (score <= -7.0)  # example cutoff
                    break
        except Exception as e:
            print(f"Docking failed for idx {idx}: {e}")
else:
    print("GNINA not found; skipping docking.")

print(f"Docking hits: {df_valid['DockHit'].sum()}")

GNINA not found; skipping docking.
Docking hits: 0


In [ ]:
# 6. ADMET filtering (Lipinski) and scaffold analysis
def lipinski(r): return (r["MW"]<=500 and r["LogP"]<=5 and r["HBD"]<=5 and r["HBA"]<=10)
df_valid["ADMET"] = df_valid.apply(lipinski, axis=1)
# Union of 2D, 3D, Docking
df_valid["UnionHit"] = df_valid[["Hit2D","Hit3D","DockHit"]].any(axis=1)
hits_final = df_valid[df_valid["UnionHit"] & df_valid["ADMET"]]
print(f"Final hits after ADMET: {len(hits_final)}")

# Scaffold counts
scf = {}
for smi in hits_final[smiles_col]:
    m = Chem.MolFromSmiles(smi)
    sc = MurckoScaffold.GetScaffoldForMol(m)
    sc_smi = Chem.MolToSmiles(sc, isomericSmiles=False)
    scf[sc_smi] = scf.get(sc_smi, 0) + 1
print("Top scaffolds:", sorted(scf.items(), key=lambda x: x[1], reverse=True)[:5])

Final hits after ADMET: 163
Top scaffolds: [('C1=CCCCC1', 37), ('O=C1C=CCCC1', 22), ('C1CCC2OC2C1', 14), ('', 11), ('c1ccccc1', 6)]


In [ ]:
# 7. Save results & visualizations
df_valid.to_csv(os.path.join(output_dir, "full_results.csv"), index=False)
hits_final.to_csv(os.path.join(output_dir, "final_hits.csv"), index=False)
print("Saved full and final hits CSV.")

# Plots
plt.figure(figsize=(6,4))
plt.hist(df_valid["Sim3D"], bins=20)
plt.title("3D Similarity Distribution")
plt.savefig(os.path.join(output_dir, "sim3d_dist.png"))
plt.close()

plt.figure(figsize=(5,5))
plt.scatter(df_valid["MW"], df_valid["LogP"], c="gray", alpha=0.5, label="Library")
plt.scatter(hits_final["MW"], hits_final["LogP"], c="red", label="Hits")
plt.title("MW vs LogP")
plt.legend()
plt.savefig(os.path.join(output_dir, "MW_vs_LogP.png"))
plt.close()

Saved full and final hits CSV.
